<a href="https://colab.research.google.com/github/hikmahealth/covid19countymap/blob/master/Hikma_COVID_County_Two_Round_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hikma COVID-19 Response - County Policy and Case Data Exploratory Data Analysis

License: Apache 2.0

In [1]:
import copy
import io
import json
import urllib

from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
import pandas as pd
import numpy as np

import os

import time

## Use census data to map county name to FIPS code

In [2]:
with urllib.request.urlopen("https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv") as infile:
  raw_data = infile.read()
# Deal with encoding issues in the Census CSV file.
replacements = {0xed: "í", 0xe1: "á", 0xf3: "ó", 0xf1: "ñ", 0xfc:"ü"}
for char, repl in replacements.items():
  raw_data = raw_data.replace(bytes([char]), repl.encode())
raw_census = pd.read_csv(io.BytesIO(raw_data))

In [3]:
census = raw_census[raw_census.COUNTY != 0].copy()

In [4]:
census["fips"] = census.STATE * 1000 + census.COUNTY
census.set_index("fips", inplace=True)
census["full_name"] = census.CTYNAME + ", " + census.STNAME
fips_code_mapping = {name: i for i, name in census.full_name.items()}

## Read the data from Google Sheets

In [5]:
auth.authenticate_user()
sheets_client = gspread.authorize(GoogleCredentials.get_application_default())
sheet = sheets_client.open_by_url("https://docs.google.com/spreadsheets/d/1LRqAKVYpa2uhQcwD8rragwuvU6gMLRzYupihsURC8Cg/edit#gid=519201207")

In [6]:
raw_policies = pd.DataFrame(sheet.sheet1.get_all_records())

In [7]:
auth.authenticate_user()
sheets_client = gspread.authorize(GoogleCredentials.get_application_default())
sheet = sheets_client.open_by_url("https://docs.google.com/spreadsheets/d/1QWLzdeL0hF4z4r3tsTqZoKc_dKF2YC7SLw-PJKckfAM/edit#gid=843876519")
raw_policies_rd2 = pd.DataFrame(sheet.sheet1.get_all_records())

## Clean up the county-level data

### Encode free-text column values

In [8]:
dimensions = ["school", "work", "shelter", "shelter_enforcement", "event", "testing", "transport"]

In [9]:
full_policies = raw_policies.rename(columns={
    "You are filling out the form for the following county:": "county_name",
    "Are schools closed in this county?": "school",
    "If yes, please provide the URL to information regarding school closures.": "school_url",
    "If yes, please provide the date of when this policy on school closures took or will take effect.": "school_date",
    "Are/were non-essential workplaces closed at any point? Is/was there there any language that suggests that those who do not provide essential services should work from home or stop working at any point?": "work",
    "If yes, please provide the URL to information regarding non-essential workplace closures.": "work_url",
    "If yes, provide the date of when this policy took or will take effect.": "work_date",
    "Is/was social distancing or shelter-in-place being enforced with fines or penalties in this county at any point? ": "shelter_enforcement",
    "If yes, please provide the URL to information regarding fine or penalty enforcement.": "shelter_enforcement_url",
    "If yes, please provide the date of when this fine or penalty enforcement took or will take effect.": "shelter_enforcement_date",
    "Is/were there shelter-in-place or stay-at-home orders in this county at any point? For example, were people advised to maintain social distance or abstain from all physical interactions or non-essential trips whenever possible?": "shelter",
    "If yes, please provide the URL to information regarding the shelter-in-place or social distancing policy.": "shelter_url",
    "If yes, please provide the date of when this shelter-in-place or social distancing policy took or will take effect.": "shelter_date",
    "Is/was there any order that public events should be restricted in size or outright cancelled at any point?": "event",
    "If yes, please provide the URL to information regarding public events restrictions.": "event_url",
    "If yes, please provide the date of when the cancellation of public events was or will be in effect.": "event_date",
    "Are there public facilities with COVID testing available in this county? ": "testing",
    "If yes, please provide the URL to information regarding testing facilities.": "testing_url",
    "If yes, please provide the date of when these facilities became available.": "testing_date",
    "Look for any information about whether this county's  buses, subways, light rail, etc. are/were shut down at any point. Is/was any public transit system shut down at any point?": "transport",
    "If yes, please provide the URL to information regarding public transit.": "transport_url",
    "If yes, please provide the date of when public transit was or will be stopped.": "transport_date",
})
full_policies.pop("Are there travel restrictions in this county? For example, are people required to self-quarantine when arriving into this county?")
full_policies.pop("Are airports closed in this county? ")
full_policies["fips"] = full_policies.county_name.map(fips_code_mapping)
full_policies.set_index("fips", inplace=True)
full_policies["updated"] = pd.to_datetime(full_policies.pop("Timestamp"))
for col in dimensions:
  full_policies[col] = full_policies[col].map({"Yes": True, "No": False, "":float('nan')})
# Convert timestamp from ambiguous DD/MM/YYYY to ISO 8601 friendly YYYY-MM-DD
for col in dimensions:
  full_policies[col + "_date"] = pd.to_datetime(full_policies[col + "_date"]).dt.strftime("%Y-%m-%d")

In [10]:
dims_rd2 = ['work', 'shelter']
full_policies_rd2 = raw_policies_rd2.rename(columns={
    "You are filling out the form for the following county:": "county_name",
    "Are all non-essential workplaces still closed? If a previous policy has been lifted and non-essential businesses have reopened to any extent please select \"No\", otherwise select \"Yes\"": "work",
    "Please provide the URL to information regarding reopening of non-essential workplaces.": "work_url",
    "If No, provide the date non-essential businesses reopened. If Yes, provide the date you are completing this form.": "work_date",
    "Is there still a shelter-in-place or stay-at-home order in this county? (E.g., are restaurants still closed/people still being advised to abstain from all physical interactions?) If a previous policy has been lifted to any extent, please select \"No\", otherwise select \"Yes\"": "shelter",
    "Please provide the URL to information regarding the lifting of the shelter-in-place or social distancing policy.": "shelter_url",
    "If No, please provide the date of when this shelter-in-place or social distancing policy was lifted. If Yes, provide the date you are completing this form.": "shelter_date",
})

full_policies_rd2["fips"] = full_policies_rd2.county_name.map(fips_code_mapping)
full_policies_rd2.set_index("fips", inplace=True)
full_policies_rd2["updated"] = pd.to_datetime(full_policies_rd2.pop("Timestamp"))
for col in dims_rd2:
  full_policies_rd2[col] = full_policies_rd2[col].map({"Yes": True, "No": False, "":float('nan')})
# Convert timestamp from ambiguous DD/MM/YYYY to ISO 8601 friendly YYYY-MM-DD
for col in dims_rd2:
  full_policies_rd2[col + "_date"] = pd.to_datetime(full_policies_rd2[col + "_date"]).dt.strftime("%Y-%m-%d")

In [11]:
full_policies_all = pd.concat([full_policies,full_policies_rd2])

### Deduplicate rows.

Naive assumption - we assume that restrictions are never lifted, so the heaviest restriction reported for fips is still active.

We will likely have to revisit this in the future.

In [12]:
full_policies = full_policies_all

with open("all_county_policies.csv", "w") as outfile:
  outfile.write(full_policies.to_csv())

In [13]:
# JS map not aligned to 'City' or 'Municipality' for now 
# policies_noloc = full_policies[~full_policies.county_name.str.contains('city') & ~full_policies.county_name.str.contains('Municipality')].copy()
# JS map can handle the data with missing FIPS codes, so including other non-county locations for now
# policies_noloc = full_policies
# Only deduplicating and keeping first entry for now, while debugging groupby
# policies_min = full_policies.groupby(by="fips").agg(func=max)
# policies_min = full_policies.groupby('fips').min().reset_index()

policies_dedup = full_policies.drop_duplicates(subset="county_name").reset_index()

# Remove if NA for FIPS (if submitter of data changed county name and FIPS could not be correctly mapped
policies_min = policies_dedup[~policies_dedup.fips.isna() & (policies_dedup.fips < 100000) & (policies_dedup.fips >= 0)].copy()

policies_min2 = policies_min

In [14]:
policies_min2.set_index('fips', inplace=True);
# full_policies.groupby('fips')['updated'].sum()

In [15]:
# function that has the following aggregation behavior:
# pick the most recently submitted entry on any particular policy, inherit latest policy for each policy 
# EXCEPT if 2 or more within 24 hours of the last entered policy
# in the case of 2 entries with disagreement, pick the one that is most open (i.e. favor No over Yes)
# if 3+ entries within 24 hours then take majority vote, leaning No in case of tie

def agg24(df):
  polcols = dimensions
  # one entry should be returned for each county, initialize to first entry and edit later
  findf = df.head(1)
  for col in polcols:
    # find last non-null datapoint for that policy
    filldf = df[~df[col].isnull()]
    tmax = filldf.updated.max()
    tcut = tmax - np.timedelta64(24,'h')
    reduced = filldf.drop(filldf[filldf.updated < tcut].index)

    # will return date and URL of the last entry that matched the final boolean
    urlcol = col + '_url'
    datecol = col + '_date'
    
    if reduced[col].size == 0:
       decision = float('nan')
    elif np.sum(reduced[col])/reduced[col].size > 0.50001:
      decision = True
    else:
      decision = False
    findf[col].iloc[0] = decision
    sourced = reduced[reduced[col] == decision]

    if sourced[col].shape[0] > 0:
      # print(sourced.shape[0])
      findf[urlcol].iloc[0] = sourced[urlcol].iloc[sourced.shape[0]-1]
      findf[datecol].iloc[0] = sourced[datecol].iloc[sourced.shape[0]-1]

 #   if reduced[col].isna().all()
  return findf
  

In [16]:
policies_agg = full_policies.groupby('fips').apply(agg24)

In [17]:
policies_agg = policies_agg.droplevel(1)

In [18]:
policies_agg.head()

,county_name,school,school_url,school_date,work,work_url,work_date,shelter_enforcement,shelter_enforcement_url,shelter_enforcement_date,shelter,shelter_url,shelter_date,event,event_url,event_date,testing,testing_url,testing_date,transport,transport_url,transport_date,updated
fips,,,,,,,,,,,,,,,,,,,,,,,
1001,"Autauga County, Alabama",True,https://whnt.com/news/coronavirus/state-of-eme...,2020-03-18,False,https://governor.alabama.gov/assets/2020/04/Sa...,2020-04-30,NaN,,NaN,True,https://www.alsde.edu/COVID19%20Updates/Alabam...,2020-03-19,True,https://www.alsde.edu/COVID19%20Updates/Alabam...,2020-03-19,True,https://www.alabamapublichealth.gov/news/2020/...,2020-05-01,NaN,,NaN,2020-05-10 02:47:49
1003,"Baldwin County, Alabama",True,https://www.bcbe.org/Page/22126,2020-03-21,NaN,https://www.alabamapublichealth.gov/legal/asse...,2020-05-11,False,,NaN,NaN,https://www.alabamapublichealth.gov/legal/asse...,2020-05-11,True,https://www.alabamapublichealth.gov/legal/asse...,2020-05-11,True,https://alpublichealth.maps.arcgis.com/apps/op...,2020-04-23,False,,NaN,2020-05-18 23:39:05
1005,"Barbour County, Alabama",True,https://governor.alabama.gov/assets/2020/03/Al...,2020-03-19,False,https://alabamapublichealth.gov/legal/assets/o...,2020-04-30,False,,NaN,True,https://governor.alabama.gov/assets/2020/04/Fi...,2020-04-04,True,https://governor.alabama.gov/assets/2020/03/Al...,2020-03-19,False,https://www.mainstreetfamilycare.com/coronavir...,NaN,False,,NaN,2020-05-23 18:51:26
1007,"Bibb County, Alabama",True,https://governor.alabama.gov/assets/2020/03/Al...,2020-03-19,False,https://www.alabamapublichealth.gov/legal/asse...,2020-04-30,False,,NaN,True,https://governor.alabama.gov/assets/2020/04/Fi...,2020-04-04,True,https://governor.alabama.gov/assets/2020/03/Al...,2020-03-19,True,http://alabamapublichealth.gov/news/2020/05/11...,2020-05-11,False,,NaN,2020-05-23 19:22:13
1009,"Blount County, Alabama",True,https://whnt.com/news/covid-19/covid-19-impact...,2020-03-16,False,https://governor.alabama.gov/assets/2020/04/Sa...,2020-04-30,NaN,,NaN,True,https://www.alsde.edu/COVID19%20Updates/Alabam...,2020-03-17,True,https://www.alsde.edu/COVID19%20Updates/Alabam...,2020-03-17,True,https://www.alabamapublichealth.gov/news/2020/...,2020-04-22,NaN,,NaN,2020-05-10 02:14:32


In [19]:
with open("county_policies.csv", "w") as outfile:
  outfile.write(policies_agg.to_csv())

  

In [24]:
full_policies.shape

(2301, 23)

In [23]:
policies_agg.shape

(1164, 23)

In [25]:
## NYT Data Import

raw_county_data = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv")
raw_state_data = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv")

with urllib.request.urlopen("https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv") as infile:
  raw_data = infile.read()
# Deal with encoding issues in the Census CSV file.
replacements = {0xed: "í", 0xe1: "á", 0xf3: "ó", 0xf1: "ñ", 0xfc:"ü"}
for char, repl in replacements.items():
  raw_data = raw_data.replace(bytes([char]), repl.encode())
raw_census = pd.read_csv(io.BytesIO(raw_data))
census = raw_census.copy()
census["fips"] = census.STATE * 1000 + census.COUNTY
census.set_index("fips", inplace=True)


def get_latest(raw_data):
  # Filter out data with missing FIPS codes
  data = raw_data[(raw_data.fips < 100000) & (raw_data.fips >= 0)].copy()
  data.fips = data.fips.astype(int)
  data = data[["fips", "cases", "deaths", "date"]].copy()
  # data.set_index("fips", inplace=True)
  return data.groupby(by="fips").agg(func=max)



county_data = get_latest(raw_county_data)
state_data = get_latest(raw_state_data)


In [26]:

# NY Times lumps all NYC together, we extrapolate by pop

NYC_COUNTIES = [
                36061,  # New York County
                36047,  # Kings County
                36005,  # Bronx County
                36085,  # Richmond County
                36081,  # Queens County
]


nycensus = census[census.index.isin(NYC_COUNTIES)]
nyc_proportions = nycensus.CENSUS2010POP / nycensus.CENSUS2010POP.sum()
nyc_data = raw_county_data[raw_county_data.county == "New York City"].agg(func=max)[["cases", "deaths", "date"]]
nyc_scaled = {}
for fips, proportion in nyc_proportions.items():
  nyc_scaled[fips] = {
      "cases": round(proportion * nyc_data.cases),
      "deaths": round(proportion * nyc_data.deaths),
      "date": nyc_data.date,
  }
county_data = county_data.append(pd.DataFrame.from_dict(nyc_scaled, orient="index"))


county_data["pop"] = census.CENSUS2010POP[county_data.index]

state_populations = census[census.COUNTY == 0].set_index("STATE").CENSUS2010POP
state_populations[60] = 56079  # American Samoa
state_populations[66] = 159358  # Guam
state_populations[69] = 53971  # Northern Mariana Islands
state_populations[72] = 3725789  # Puerto Rico
state_populations[78] = 106405  # Virgin Islands

state_data["pop"] = state_populations

with open("county_cases.json", "w") as outfile:
  outfile.write(county_data.to_json(orient="index"))

with open("state_cases.json", "w") as outfile:
  outfile.write(state_data.to_json(orient="index"))

In [27]:
#county_data['fips'] = county_data.index

In [28]:
policies_agg = policies_agg.join(county_data, on='fips' )

In [29]:
with open("county_policies.csv", "w") as outfile:
  outfile.write(policies_agg.to_csv())

with open("county_policies.json", "w") as outfile:
  outfile.write(policies_agg.to_json(orient="index"))

In [30]:
policies_agg.head()

,county_name,school,school_url,school_date,work,work_url,work_date,shelter_enforcement,shelter_enforcement_url,shelter_enforcement_date,shelter,shelter_url,shelter_date,event,event_url,event_date,testing,testing_url,testing_date,transport,transport_url,transport_date,updated,cases,deaths,date,pop
fips,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,"Autauga County, Alabama",True,https://whnt.com/news/coronavirus/state-of-eme...,2020-03-18,False,https://governor.alabama.gov/assets/2020/04/Sa...,2020-04-30,NaN,,NaN,True,https://www.alsde.edu/COVID19%20Updates/Alabam...,2020-03-19,True,https://www.alsde.edu/COVID19%20Updates/Alabam...,2020-03-19,True,https://www.alabamapublichealth.gov/news/2020/...,2020-05-01,NaN,,NaN,2020-05-10 02:47:49,503.0,12.0,2020-06-28,54571.0
1003,"Baldwin County, Alabama",True,https://www.bcbe.org/Page/22126,2020-03-21,NaN,https://www.alabamapublichealth.gov/legal/asse...,2020-05-11,False,,NaN,NaN,https://www.alabamapublichealth.gov/legal/asse...,2020-05-11,True,https://www.alabamapublichealth.gov/legal/asse...,2020-05-11,True,https://alpublichealth.maps.arcgis.com/apps/op...,2020-04-23,False,,NaN,2020-05-18 23:39:05,575.0,10.0,2020-06-28,182265.0
1005,"Barbour County, Alabama",True,https://governor.alabama.gov/assets/2020/03/Al...,2020-03-19,False,https://alabamapublichealth.gov/legal/assets/o...,2020-04-30,False,,NaN,True,https://governor.alabama.gov/assets/2020/04/Fi...,2020-04-04,True,https://governor.alabama.gov/assets/2020/03/Al...,2020-03-19,False,https://www.mainstreetfamilycare.com/coronavir...,NaN,False,,NaN,2020-05-23 18:51:26,317.0,1.0,2020-06-28,27457.0
1007,"Bibb County, Alabama",True,https://governor.alabama.gov/assets/2020/03/Al...,2020-03-19,False,https://www.alabamapublichealth.gov/legal/asse...,2020-04-30,False,,NaN,True,https://governor.alabama.gov/assets/2020/04/Fi...,2020-04-04,True,https://governor.alabama.gov/assets/2020/03/Al...,2020-03-19,True,http://alabamapublichealth.gov/news/2020/05/11...,2020-05-11,False,,NaN,2020-05-23 19:22:13,162.0,1.0,2020-06-28,22915.0
1009,"Blount County, Alabama",True,https://whnt.com/news/covid-19/covid-19-impact...,2020-03-16,False,https://governor.alabama.gov/assets/2020/04/Sa...,2020-04-30,NaN,,NaN,True,https://www.alsde.edu/COVID19%20Updates/Alabam...,2020-03-17,True,https://www.alsde.edu/COVID19%20Updates/Alabam...,2020-03-17,True,https://www.alabamapublichealth.gov/news/2020/...,2020-04-22,NaN,,NaN,2020-05-10 02:14:32,189.0,1.0,2020-06-28,57322.0


In [31]:
# with open("county_policies.json", "w") as outfile:
#   outfile.write(policies_agg.to_json(orient="index"))

In [32]:
# with open("state_cases.csv", "w") as outfile:
#   outfile.write(state_data.to_csv())

In [33]:
# Now includes the NYT case data in policies_agg
policies_agg.shape

(1164, 27)